In [1]:
import pandas as pd 
import numpy as np 
from collections import defaultdict
import re

# Cleaning function:

In [2]:
def preprocess_string(str_arg):
    
    cleaned_str=re.sub('[^a-z\s]+',' ',str_arg,flags=re.IGNORECASE) 
    cleaned_str=re.sub('(\s+)',' ',cleaned_str) 
    cleaned_str=cleaned_str.lower() 
    
    return cleaned_str 

# Naive Bayes from scratch:

In [3]:
class NaiveBayes:
    
    def __init__(self,unique_classes):
        
        self.classes=unique_classes # Constructor is sinply passed with unique number of classes of the training set
        

    def addToBow(self,example,dict_index):
        
        
        if isinstance(example,np.ndarray): example=example[0]
     
        for token_word in example.split(): #for every word in preprocessed example
          
            self.bow_dicts[dict_index][token_word]+=1 #increment in its count
            
    def train(self,dataset,labels):
        
    
        self.examples=dataset
        self.labels=labels
        self.bow_dicts=np.array([defaultdict(lambda:0) for index in range(self.classes.shape[0])])
        
        #only convert to numpy arrays if initially not passed as numpy arrays - else its a useless recomputation
        
        if not isinstance(self.examples,np.ndarray): self.examples=np.array(self.examples)
        if not isinstance(self.labels,np.ndarray): self.labels=np.array(self.labels)
            
        #constructing BoW for each category
        for cat_index,cat in enumerate(self.classes):
          
            all_cat_examples=self.examples[self.labels==cat] #filter all examples of category == cat
            
            #get examples preprocessed
            
            cleaned_examples=[preprocess_string(cat_example) for cat_example in all_cat_examples]
            
            cleaned_examples=pd.DataFrame(data=cleaned_examples)
            
            #now costruct BoW of this particular category
            np.apply_along_axis(self.addToBow,1,cleaned_examples,cat_index)
            
                
      
        prob_classes=np.empty(self.classes.shape[0])
        all_words=[]
        cat_word_counts=np.empty(self.classes.shape[0])
        for cat_index,cat in enumerate(self.classes):
           
            #Calculating prior probability p(c) for each class
            prob_classes[cat_index]=np.sum(self.labels==cat)/float(self.labels.shape[0]) 
            
            #Calculating total counts of all the words of each class 
            count=list(self.bow_dicts[cat_index].values())
            cat_word_counts[cat_index]=np.sum(np.array(list(self.bow_dicts[cat_index].values())))+1 # |v| is remaining to be added
            
            #get all words of this category                                
            all_words+=self.bow_dicts[cat_index].keys()
                                                     
        
        #combine all words of every category & make them unique to get vocabulary -V- of entire training set
        
        self.vocab=np.unique(np.array(all_words))
        self.vocab_length=self.vocab.shape[0]
                                  
        #computing denominator value                                      
        denoms=np.array([cat_word_counts[cat_index]+self.vocab_length+1 for cat_index,cat in enumerate(self.classes)])                                                                          
      
        
        self.cats_info=[(self.bow_dicts[cat_index],prob_classes[cat_index],denoms[cat_index]) for cat_index,cat in enumerate(self.classes)]                               
        self.cats_info=np.array(self.cats_info)                                 
                                              
                                              
    def getExampleProb(self,test_example):                                
                                             
                                              
        likelihood_prob=np.zeros(self.classes.shape[0]) #to store probability w.r.t each class
        
        #finding probability w.r.t each class of the given test example
        for cat_index,cat in enumerate(self.classes): 
                             
            for test_token in test_example.split(): #split the test example and get p of each test word
                                             
                
                #get total count of this test token from it's respective training dict to get numerator value                           
                test_token_counts=self.cats_info[cat_index][0].get(test_token,0)+1
                
                #now get likelihood of this test_token word                              
                test_token_prob=test_token_counts/float(self.cats_info[cat_index][2])                              
                
                #remember why taking log? To prevent underflow!
                likelihood_prob[cat_index]+=np.log(test_token_prob)
                                              
        # we have likelihood estimate of the given example against every class but we need posterior probility
        post_prob=np.empty(self.classes.shape[0])
        for cat_index,cat in enumerate(self.classes):
            post_prob[cat_index]=likelihood_prob[cat_index]+np.log(self.cats_info[cat_index][1])                                  
      
        return post_prob
    
   
    def test(self,test_set):
            
       
        predictions=[] #to store prediction of each test example
        for example in test_set: 
                                              
            #preprocess the test example the same way we did for training set exampels                                  
            cleaned_example=preprocess_string(example) 
             
            #simply get the posterior probability of every example                                  
            post_prob=self.getExampleProb(cleaned_example) #get prob of this example for both classes
            
            #simply pick the max value and map against self.classes!
            predictions.append(self.classes[np.argmax(post_prob)])
                
        return np.array(predictions) 

# Training data-set:

In [4]:
training_set=pd.read_csv ("train.csv")
training_set.info()
training_set.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         60000 non-null  int64 
 1   comment    60000 non-null  object
 2   subreddit  60000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB


(60000, 3)

In [5]:
training_set.head()

,id,comment,subreddit
0,0,I think prestige points should not expire ever...,leagueoflegends
1,1,Whats going to happen with them if they will b...,europe
2,2,Anecdotal evidence is anecdotal. Clearly by “e...,gameofthrones
3,3,"Look dude, with all due respect, your music is...",Music
4,4,Hope he gets the doomhammer back!,wow


# Prediction on train data-set:

In [6]:
#getting training set examples labels
y_train=training_set['subreddit'].values
x_train=training_set['comment'].values


from sklearn.model_selection import train_test_split
train_data,test_data,train_labels,test_labels=train_test_split(x_train,y_train,
                                                               shuffle=True,
                                                               test_size=0.2
                                                               ,random_state=42
                                                               ,stratify=y_train)
classes=np.unique(train_labels)

# Training phase....

nb=NaiveBayes(classes)
nb.train(train_data,train_labels)

# Testing phase 

y_pred_train=nb.test(test_data)
test_acc=np.sum(y_pred_train==test_labels)/float(test_labels.shape[0])

print ("Test Set Accuracy: ",test_acc) 

Test Set Accuracy:  0.4529166666666667


# Testing data-set:

In [7]:
testing_set=pd.read_csv ("test.csv")
testing_set.info()
testing_set.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       20000 non-null  int64 
 1   comment  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 234.4+ KB


(20000, 2)

In [8]:
testing_set.head()

,id,comment
0,0,Holy shit a shot counter.
1,1,It doesn't matter that it isn't hard to rememb...
2,2,I find it funny that this is downvoted
3,3,They are really getting ridicoulous with all t...
4,4,He's Eden's best friend


# Prediction on test data-set:

In [9]:
X_test=testing_set.comment.values

y_pred_test=nb.test(X_test) 

# Submission:

In [10]:
submission = zip(list(range(len(y_pred_test))), y_pred_test)
test_df = pd.DataFrame(submission, columns=['Id','Category'])
test_df.to_csv('submission.csv', index = False, header=True)

# Naive Bayes cross validation:

In [12]:
# scores = cross_val_score(nb, train_data, train_labels, cv=5)
# scores

################################################################

# Preparin the data for selective classifiers:

In [17]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def clean_text(training_set):
    
    all_comments = list()
    lines = training_set["comment"].values.tolist()
    for text in lines:
        text = text.lower()
        
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub("", text)       
    
        
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        
        tokens = word_tokenize(text)
        
        table = str.maketrans('', '', string.punctuation)
        
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        
        words = [w for w in words if not w in stop_words]
        words = ' '.join(words)
        
        all_comments.append(words)
    return all_comments

all_comments = clean_text(training_set)
all_comments[0:2]

['think prestige points not expire ever skins buy available set duration exemple year release another skin vault old one making also limitededition skin also please love god nt rerelease skins need grind prestige shop would suck everyone grinded',
 'whats going happen refused asilum appeal']

# Most frequent used words

In [ ]:
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer

c = all_comments
filtered_sentence = [] 
freq_count_limit = FreqDist()
lemmatizer=WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

for i in c:
    comment_tokens = word_tokenize(i)
    
    for words in comment_tokens:
        if words not in stop_words: 
            filtered_sentence.append(words) 
        
            limit_words = lemmatizer.lemmatize(words)
#     for word in root_words:
            freq_count_limit[limit_words.lower()]+=1
freq_count_limit

In [ ]:
import matplotlib.pyplot as plt

freq_count_limit.plot(30,cumulative=False)
plt.show()

# Vectorizing and transforming the text:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(ngram_range=(1,1), max_features=30000, strip_accents='ascii')
vect.fit(all_comments)
vocabulaire = vect.get_feature_names()

In [ ]:
bag_of_words = vect.transform(all_comments)
bag_of_words.shape

# Random Forest:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(max_depth=15, random_state=42)
clf_rf.fit(bag_of_words, training_set['subreddit'])

In [ ]:
clf_rf.score(bag_of_words, training_set['subreddit'])

### Random Forest - Cross Validation:

In [ ]:
from sklearn.model_selection import cross_val_score

scores_rf = cross_val_score(clf_rf, bag_of_words, training_set['subreddit'], cv=5)
scores_rf

# Logistic Regression:

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression(max_iter=15, random_state=42)
clf_lr.fit(bag_of_words, training_set['subreddit'])

### Logistic Regression - Cross Validation:

In [ ]:
clf_lr.score(bag_of_words, training_set['subreddit'])

### Logistic Regression - Cross Validation:

In [ ]:
scores_lr = cross_val_score(clf_lr, bag_of_words, training_set['subreddit'], cv=5)
scores_lr